In [1]:
source ../config_duke.sh -v

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect



In [4]:
ls -d ${FD_RES}/count_fragment/*/target_PER1.bed.gz

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1_20x/target_PER1.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1/target_PER1.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input2_20x/target_PER1.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input2/target_PER1.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input3_20x/target_PER1.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input3/target_PER1.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input4_20x/target_PER1.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input4/target_PER1.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input5_20x/target_PER1.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input5/target_PER1.bed.gz
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/TFX2_Dex/target_

In [5]:
fpath="/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1/target_PER1.bed.gz"
zcat $fpath | head 

chr17	8148117	8149012	1
chr17	8148122	8149107	1
chr17	8148178	8149194	1
chr17	8148188	8149154	1
chr17	8148190	8149151	1
chr17	8148220	8149108	1
chr17	8148280	8149232	1
chr17	8148401	8149372	1
chr17	8148548	8149585	1
chr17	8148913	8149953	1


In [7]:
echo ${NODE}

all


In [14]:
FD_BEDS=($(ls -d ${FD_RES}/count_fragment/{Input?,Input?_20x,TFX?_DMSO,TFX?_Dex}/))
for FD_BED in ${FD_BEDS[@]}; do
   SAMPLE=$(basename ${FD_BED})
   FD_OUT=${FD_RES}/nuc/${SAMPLE}
   
   echo ${FD_BED}
   echo ${FD_OUT}
   echo ${SAMPLE}
   
done

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/nuc/Input1
Input1
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/nuc/Input1_20x
Input1_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input2/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/nuc/Input2
Input2
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input2_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/nuc/Input2_20x
Input2_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input3/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/nuc/Input3
Input3
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input3_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/nuc/Input3_20x
Input3_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input4/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/nuc/Input

In [21]:
### set environment
source ../config_duke.sh
source ../config_load_module_bedtools.sh

sbatch -p ${NODE} \
    --mem 8G \
    -o ${FD_LOG}/nuc_fragment_per1.txt \
    <<'EOF'
#!/bin/bash

### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=${CHROMS[${SLURM_ARRAY_TASK_ID}]}
#TARGET=chr${CHROM}

CHROM=chr17
TARGET=target_PER1

### set input and output
FD_BEDS=($(ls -d ${FD_RES}/count_fragment/{Input?,Input?_20x,TFX?_DMSO,TFX?_Dex}/))
FN_BED=${TARGET}.bed.gz
FN_OUT=${TARGET}.bed.gz

### genome annotation
FD_GEN=${FD_ANN}/genome/hg38/fasta
FN_GEN=${CHROM}.fa

### START Message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    FD_OUT=${FD_RES}/nuc/${SAMPLE}
    
    ### Loop START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Output file: " ${FD_OUT}/${FN_OUT}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    zcat ${FD_BED}/${FN_BED} | head -n 3
    echo
    
    ### init: create output folder if not exist
    mkdir -p ${FD_OUT}
    
    ### calculate the GC content of each fragment
    bedtools nuc \
        -fi  ${FD_GEN}/${FN_GEN} \
        -bed ${FD_BED}/${FN_BED} |\
        gzip -c > ${FD_OUT}/${FN_OUT}
    
    ### Loop END Message
    echo "Show the first few lines of the output file"
    echo ${FD_OUT}/${FN_OUT}
    zcat ${FD_OUT}/${FN_OUT} | head -n 3
    echo
done

### END message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 27247358


In [ ]:
cat ${FD_LOG}/nuc_fragment_per1.txt